In [9]:
from cnn_model import *
import sys
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

def cross_entropy(y_predict, label):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_predict, labels=label))
    #優化器: 使cross entropy最小化
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    #預測準確度
    #argmax: 得到Tensor中最大值的index
    correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(label,1))
    #神經網路準確率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return train_step, accuracy




model = mnist_cnn_model()
y = model.run_cnn()


train_step, accuracy = cross_entropy(y, model.label_placeholder)
train_accuracy = 0.0



#initialize
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

#train for 1000 times
for i in range(200):
    batch = mnist.train.next_batch(100)
    #跑訓練集，利用optimizer將結果導到較好的地方
    train_step.run(session=sess, feed_dict={model.img_placeholder: batch[0],
                                            model.label_placeholder: batch[1], model.keep_prob: 0.7})
    
    if i%10 == 0:
        #輸出準確度
        train_accuracy = accuracy.eval(session=sess, feed_dict={model.img_placeholder:batch[0], 
                                        model.label_placeholder: batch[1], model.keep_prob: 1.0})
        
        print("step {}, training accuracy {:}".format(i, train_accuracy))

print("Accuracy: ", sess.run(accuracy, feed_dict={model.img_placeholder: mnist.test.images,
                                model.label_placeholder: mnist.test.labels, model.keep_prob: 1.0}))





print("test:", sess.run(tf.argmax(mnist.test.labels,1)))
print("model:", sess.run(tf.argmax(y, 1), feed_dict={model.img_placeholder: mnist.test.images,
                                            model.label_placeholder: mnist.test.labels, 
                                            model.keep_prob: 1.0}))


# Saving
'''
import pickle
export_dir =  "my_net/save_net.pickle"
model.details()
with open(export_dir, "wb") as f:
    pickle.dump(model, f)
    print("Model saved in path: ", export_dir)


'''
export_dir =  "my_net/save_net.ckpt"
# saver = tf.train.Saver()
saver = tf.train.Saver({
            "W_conv1": model.W_conv1, 
            "W_conv2": model.W_conv2, 
            "b_conv1": model.b_conv1, 
            "b_conv2": model.b_conv2, 
            "W_fc1": model.W_fc1, 
            "W_fc2": model.W_fc2, 
            "b_fc1": model.b_fc1, 
            "b_fc2": model.b_fc2
        })
sess.run(init)
save_path = saver.save(sess, export_dir)
#save_path = saver.save(sess, export_dir, global_step = 200)
print("Model saved in path: ", save_path)


def details(model, sess):
    print("{:30}:{}".format("convolution weight 1", sess.run(model.W_conv1)))
    #print("{:30}:{}".format("convolution bias 1", sess.run(model.b_conv1)))
    print("{:30}:{}".format("convolution weight 2", sess.run(model.W_conv2)))
    #print("{:30}:{}".format("convolution bias 2", sess.run(model.b_conv2)))

    print("{:30}:{}".format("full connected weight 1", sess.run(model.W_fc1)))
    #print("{:30}:{}".format("full connected bias 1", sess.run(model.b_fc1)))
    print("{:30}:{}".format("full connected weight 2", sess.run(model.W_fc2)))
    #print("{:30}:{}".format("full connected bias 2", sess.run(model.b_fc2)))
    return

print(sess.run(y , {model.img_placeholder: mnist.test.images,
                                            model.label_placeholder: mnist.test.labels, 
                                            model.keep_prob: 1.0}))


step 0, training accuracy 0.14000000059604645
step 10, training accuracy 0.20000000298023224
step 20, training accuracy 0.3400000035762787
step 30, training accuracy 0.3700000047683716
step 40, training accuracy 0.5400000214576721
step 50, training accuracy 0.5099999904632568
step 60, training accuracy 0.6299999952316284
step 70, training accuracy 0.550000011920929
step 80, training accuracy 0.6899999976158142
step 90, training accuracy 0.75
step 100, training accuracy 0.7099999785423279
step 110, training accuracy 0.7599999904632568
step 120, training accuracy 0.7599999904632568
step 130, training accuracy 0.7200000286102295
step 140, training accuracy 0.7200000286102295
step 150, training accuracy 0.75
step 160, training accuracy 0.7200000286102295
step 170, training accuracy 0.6600000262260437
step 180, training accuracy 0.8100000023841858
step 190, training accuracy 0.7799999713897705
Accuracy:  0.7605
test: [7 2 1 ... 4 5 6]
model: [7 2 1 ... 9 8 6]
Model saved in path:  my_net/sa